In [1]:
from examples.gpt4v.instruct import system_prompt
from examples.gpt4v.oai import chat
from agentdesk import Desktop

In [ ]:
# Create a local desktop
desktop = Desktop.local()
desktop.view(background=True)

In [2]:
from agentdesk.tool import Desktop

desk = Desktop(agentd_url="http://localhost:8000")

connected to desktop via agentd


In [4]:
# Get the actions a model can take on the desktop as json schema
actions = desk.json_schema()
actions

[{'name': 'click',
  'parameters': {'type': 'object',
   'properties': {'button': {'type': 'string',
     'description': 'Button to click. Defaults to "left".'},
    'x': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
     'description': 'X coordinate to move to, if not provided it will click on current location. Defaults to None.'},
    'y': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
     'description': 'Y coordinate to move to, if not provided it will click on current location. Defaults to None.'}},
   'required': []},
  'description': 'Click mouse button'},
 {'name': 'double_click',
  'parameters': {'type': 'object', 'properties': {}, 'required': []},
  'description': 'Double click the mouse'},
 {'name': 'drag_mouse',
  'parameters': {'type': 'object',
   'properties': {'x': {'type': 'integer', 'description': 'x coordinate'},
    'y': {'type': 'integer', 'description': 'y coordinate'}},
   'required': ['x', 'y']},
  'description': 'Drag the mouse'},
 {'name': 'move_mouse

In [5]:
msgs = []

In [6]:
# function calling not yet supported in gpt4v
msg =  {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt(actions)}],
}
msgs.append(msg)

In [7]:
response = chat(msgs)
msgs.append(response)
response

{'role': 'assistant',
 'content': 'I am ready. Please send me the current screenshot and mouse coordinates.'}

In [ ]:
b64_img = desktop.take_screenshot()
x, y = desktop.mouse_coordinates()

In [8]:
msg = {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"Current mouse coordinates are ({x}, {y}), and the task to solve is '{task}', please return the appropriate next action"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{b64_img}"
          }
        }
      ]
}
msgs.append(msg)